In [7]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
from openpyxl.chart import BarChart, Reference
from pandas import ExcelWriter
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font
from openpyxl.formatting.rule import ColorScaleRule 

In [8]:
#abrimos csv y convertimos a xlsx
df=pd.read_csv('players_20.csv')
writer = ExcelWriter('fifa_20.xlsx')
df.to_excel(writer, 'players_20.csv', index=True)
writer.save()

# Automatizacion
imput fifa_año.xlsx / Output reporte_año.xlsx

In [9]:
def automatizar_excel(nombrearchivo):
    
     #imput fifa_año.xlsx / Output reporte_año.xlsx
    
    df20 =pd.read_excel(nombrearchivo)
     
    #DATA
    top_ten_20=df20.sort_values(by='overall',ascending= False).head(10)
    top_ten_20.set_index('short_name', inplace = True)
    top_ten=top_ten_20[['overall','potential','power_jumping','power_long_shots']]
    
    año_extension=nombrearchivo.split('_')[1]
    
#   Data big five
    
    data=df20.pivot_table(index='club',values=[ "age","overall","value_eur", "wage_eur",'shooting', 'passing', 'dribbling',
                                         "power_jumping","power_shot_power", "power_long_shots"], aggfunc='mean')

    index=data[data['wage_eur']==0].index  #buscamos los errores de club (jugadores que no tienen pago)
    data.drop(index, inplace=True) # y borramos
    data5=data.nlargest(5,'overall')  #elegimos topfive
    dat5=np.round(data5, decimals = 2)# mostramos solo 2 digitos
    
    writer =pd.ExcelWriter(f'reporte_{año_extension}')
    
    
    top_ten.to_excel(writer, sheet_name='hoja_one')
    dat5.to_excel(writer,sheet_name='Big five')
    writer.save()
            
    #formato
    wb = load_workbook(f'reporte_{año_extension}')
    pestaña=wb['hoja_one']
    pestaña2=wb['Big five']
  
       
    
    mincol=wb.active.min_column
    maxcol=wb.active.max_column
    minrow=wb.active.min_row
    maxrow=wb.active.max_row
    
    style= Side(border_style='medium', color="00000000")
    border = Border(left=style, right=style, top=style, bottom=style)
    
    for row in pestaña['A1:E11']:
        for cell in row:
            cell.border = border
            
    for row in pestaña2['A1:k6']:
        for cell in row:
            cell.border = border        
            
    fill = PatternFill(patternType='solid',                       
                       start_color='00C0C0C0',
                       end_color='00C0C0C0')
    
    #formato condicional
    rule = ColorScaleRule(start_type='percentile', start_value=10, start_color='00FFFF00',
                          mid_type='percentile', mid_value=50, mid_color='00FF6600',
                          end_type='percentile', end_value=90, end_color='00FF0000')
    
    pestaña.conditional_formatting.add('B2:E11',rule)
    pestaña2.conditional_formatting.add('C2:I6',rule)
    pestaña2.conditional_formatting.add('B2:B6',rule)
    pestaña2.conditional_formatting.add('J2:J6',rule)
    pestaña2.conditional_formatting.add('K2:K6',rule)
    
                         
    for row in pestaña['A1:E1']:
        for cell in row:
            cell.fill = fill
            
    for row in pestaña2['A1:K1']:
        for cell in row:
            cell.fill = fill
     
     
    #insert Chart
    
    barchart= BarChart()

    data= Reference(pestaña, min_col = mincol+1, max_col=maxcol, min_row=minrow, max_row=maxrow)
    categoria= Reference(pestaña, min_col= mincol, max_col= mincol, min_row= minrow+1, max_row=maxrow)

    barchart.add_data(data, titles_from_data=True)
    barchart.set_categories(categoria)

    pestaña.add_chart(barchart,'A14')
    barchart.title='Top ten players Fifa 20'
    barchart.style=2
    
    wb.save(f'reporte_{año_extension}')

    return


In [10]:
automatizar_excel('fifa_20.xlsx')

#imput fifa_año.xlsx / Output reporte_año.xlsx